In [21]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [43]:
df = pd.read_csv('/Users/cmcglade/Documents/Projects/north-carolina-ice-arrests/processed_data/nc_ice_arrests.csv')


df = df[['Apprehension Date', 'standardized_date', 'year', 'Apprehension State', 'Apprehension AOR', 'Final Program', 'Apprehension Method',
       'Apprehension Criminality', 'Case Status', 'Case Category',
       'Departed Date', 'Departure Country', 'Final Order Yes No',
       'Final Order Date', 'Birth Year', 'Citizenship Country',
       'Gender', 'Apprehension Site Landmark', 'Unique Identifier', 'comparetime', 'county', 'in_custody_arrest', 'method_cat']]
df.columns = ['app_date', 'standardized_date', 'year', 'state', 'aor', 'final_program',
              'app_method', 'criminality',
              'case_status', 'case_cat', 'departed_date',
              'departure_country', 'final_order', 'final_order_date',
              'birth_year', 'cit_country', 'gender', 'landmark',
              'id', 'comparetime', 'county', 'in_custody_arrest','method_cat']

Immigration and Customs Enforcement arrested about three times the number of people in the first half of this year in Mecklenburg County compared to the same time last year. 

That’s a steeper increase than the rest of the state, where data shows at least 2.6 times the number of arrests compared to last year.

Statewide, ICE arrested <b>about</b> 1,940 people from Jan. 20 until the end of June. 


In [23]:
meck = df[df['county']=='mecklenburg county']

arrests = meck.groupby('comparetime').size().to_frame(name='count').reset_index()
print((749-238)/238)
print(arrests.head())

arrests = df.groupby('comparetime').size().to_frame(name='count').reset_index()
print((1937-741)/741)
print(arrests.head())

2.1470588235294117
            comparetime  count
0             last_year    238
1  not in compare range    633
2             this_year    750
1.6140350877192982
            comparetime  count
0             last_year    741
1  not in compare range   1913
2             this_year   1941


At least 60% have left or were deported.

In [27]:
departed = [
    '8-Excluded/Removed - Inadmissibility',
    '6-Deported/Removed - Deportability',
    '5-Title 50 Expulsion',
    '3-Voluntary Departure Confirmed',
    '9-VR Witnessed'
]
thisyear=df[df['comparetime']=='this_year']
print(thisyear[thisyear['case_status'].isin(departed)].shape)
print(1169/1941)


(1169, 22)
0.6022668727460072


One in three of the people arrested in the first six months were facing criminal charges but had not been convicted. 

And at least 420 people – about one in five – had no reported criminal charges or convictions, according to the data. But ICE says they had immigration violations, according to the data. 

A significant difference this year is that ICE agents arrested a larger share of people who had unresolved criminal charges before they were either cleared or convicted, according to the federal data. About 330 people were in that category during the first six months of this year.

In [34]:
arrests = df.groupby(['comparetime', 'criminality']).size().to_frame(name='arrests').reset_index()
arrests['total'] = arrests.groupby('comparetime')['arrests'].transform('sum')
arrests['perc'] = (arrests['arrests']/arrests['total'])*100
arrests = arrests[arrests['comparetime']!='not in compare range']
arrests.sort_values('criminality')

,comparetime,criminality,arrests,total,perc
0,last_year,1 Convicted Criminal,482,741,65.047233
6,this_year,1 Convicted Criminal,869,1941,44.770737
1,last_year,2 Pending Criminal Charges,101,741,13.630229
7,this_year,2 Pending Criminal Charges,648,1941,33.384853
2,last_year,3 Other Immigration Violator,158,741,21.322537
8,this_year,3 Other Immigration Violator,424,1941,21.844410


The vast majority of people arrested by ICE this year and last in North Carolina were men in their 30s from Latin American countries, with the largest shares from Mexico, Honduras and Guatemala.

In [39]:
arrests = df.groupby(['comparetime', 'cit_country']).size().to_frame(name='count').reset_index()
arrests['total'] = arrests.groupby('comparetime')['count'].transform('sum')
arrests['perc'] = (arrests['count']/arrests['total'])*100
arrests = arrests[arrests['comparetime']=='this_year']
arrests.sort_values('count',ascending=False).head(20)

,comparetime,cit_country,count,total,perc
117,this_year,MEXICO,899,1941,46.316332
107,this_year,HONDURAS,348,1941,17.928903
105,this_year,GUATEMALA,195,1941,10.046368
136,this_year,VENEZUELA,117,1941,6.027821
98,this_year,EL SALVADOR,111,1941,5.718702
119,this_year,NICARAGUA,60,1941,3.091190
91,this_year,COLOMBIA,50,1941,2.575992
96,this_year,ECUADOR,22,1941,1.133436
122,this_year,PERU,18,1941,0.927357
93,this_year,CUBA,16,1941,0.824317


In [41]:
df['approx_age'] = 2025-df['birth_year']
arrests = df.groupby('comparetime')['approx_age'].mean().to_frame(name='avg_age').reset_index()
arrests.head()

,comparetime,avg_age
0,last_year,35.987854
1,not in compare range,36.768949
2,this_year,35.060793


For those people who The Observer confirmed were arrested in North Carolina during both periods, more than half were arrested straight from county jails and state or federal prisons.

In [45]:
arrests = df.groupby(['comparetime', 'method_cat']).size().to_frame(name='count').reset_index()
arrests['total'] = arrests.groupby('comparetime')['count'].transform('sum')
arrests['perc'] = (arrests['count']/arrests['total'])*100
arrests = arrests[arrests['comparetime']!='not in compare range']
#arrests.to_csv('/content/drive/MyDrive/ice_data/arrests-by-method-cat-nc.csv')
arrests.sort_values('method_cat').head(20)

,comparetime,method_cat,count,total,perc
0,last_year,287g,51,741,6.882591
12,this_year,287g,183,1941,9.428130
1,last_year,"federal, state or local incarceration",391,741,52.766532
13,this_year,"federal, state or local incarceration",1083,1941,55.795981
2,last_year,"non-custodial, located, traffic check or works...",201,741,27.125506
14,this_year,"non-custodial, located, traffic check or works...",555,1941,28.593509
3,last_year,other,3,741,0.404858
15,this_year,other,46,1941,2.369912
4,last_year,probation and parole,94,741,12.685560
16,this_year,probation and parole,67,1941,3.451829


NOTE* three people in both time frames were arrested twice. This does not affect percentages.